# FAST-UAV - Uncertainty analysis including both custom and off-the-shelf components

The purpose of this notebook is to run uncertainty analyses in order to evaluate the impacts of models uncertainty and input parameters uncertainty.

## 1. Setting up the deterministic problem

In a first place, we define in the configuration file the UAV models that will be used to run the analyses. Here, no optimization of the design is carried out, i.e. we start from an existing design and analyse its performance on a given mission.

In [1]:
import os.path as pth
import openmdao.api as om
import logging
import shutil
import fastoad.api as oad
from time import time
from fastuav.utils.drivers.cmaes_driver import CMAESDriver
import cma
import matplotlib.pyplot as plt
from fastuav.utils.postprocessing.sensitivity_analysis.sensitivity_analysis import (
    morris_analysis,
    sobol_analysis,
)

plt.rcParams["figure.figsize"] = 16, 8
plt.rcParams.update({"font.size": 13})

DATA_FOLDER_PATH = "../data"
CONFIGURATIONS_FOLDER_PATH = "../configurations"
WORK_FOLDER_PATH = "./workdir"

CONFIGURATION_FILE_MDA = pth.join(WORK_FOLDER_PATH, "multirotor_mda_cots.yaml")  # MDA Problem (i.e., with consistency constraints solving)
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "problem_outputs_discrete_DJI_M600_mdo.xml")

# For having log messages display on screen
# logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

C:\Users\f.pollet.ISAE-SUPAERO\AppData\Local\Temp\ipykernel_15900\1871020117.py:29: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Let's start by defining the multidisciplinary analysis (MDA). The [MDA](https://websites.isae-supaero.fr/IMG/pdf/2016_mca_martins_mdo_course_slides.pdf) consists of evaluating the model while solving the coupled equations.<br>
With a NVH formulation as adopted in FAST-UAV, the MDA is carried out by the optimizer instead of a dedicated solver. The optimizer is responsible for solving the consistency constraints, that is the coupled states of the system. For more information about the consistency constraints please refer to Delbecq et al. article available [here](https://oatao.univ-toulouse.fr/26691/).

In [2]:
oad.generate_configuration_file(
    CONFIGURATION_FILE_MDA, overwrite=True, distribution_name="fastuav", sample_file_name="multirotor_mda_cots.yaml"
)

'D:\\THESE\\Tools\\FAST-UAV\\src\\fastuav\\notebooks\\workdir\\multirotor_mda_cots.yaml'

In [3]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE_MDA, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width="100%", height="500px")

We now load an existing UAV design on which the uncertainty analysis will be run.

In [4]:
oad.variable_viewer(SOURCE_FILE)

To make sure that the source file is consistent with the problem definition and the models, an MDA is carried out. No difference between the values of the source file and the output values of the problem should be observed.<br>

In [5]:
INPUT_FILE = oad.generate_inputs(CONFIGURATION_FILE_MDA, SOURCE_FILE, overwrite=True)
problem = oad.optimize_problem(
    CONFIGURATION_FILE_MDA, overwrite=True
)  # Run MDA
REFERENCE_DESIGN_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
oad.variable_viewer(REFERENCE_DESIGN_FILE)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5490026273154804
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1
Optimization Complete
-----------------------------------


## 2. Design of Experiments

In contrast with a single problem evaluation, the uncertainty analysis relies on multiple evaluations of the problem to analyse the effects of varying one or several parameters on the output variables of interest.

The [OpenTurns](https://openturns.github.io/openturns/latest/index.html) and [SALib](https://salib.readthedocs.io/en/latest/index.html) libraries are used to set up the DoEs and run the sensitivity analyses. 
The drivers to connect these librairies to FAST-UAV are provided by Onera's [OpenMDAO Extensions](https://github.com/OneraHub/openmdao_extensions).

## 2.1 Screening procedure - Morris method

The Morris method allows to get a measure of importance and interaction of input factors. It is used as a screening method to reduce the number of parameters prior to a more detailed analysis.

The `morris_analysis` function takes the following inputs:
- The configuration file of the problem to be evaluated;
- The source file containing the reference design, to help the user to set up the analysis (uncertain parameters selection, output variable of interest, ...). 

The number of simulation runs required are $r(k+1)$ where $r$ is the number of trajectories (typically between 4 to 10) and $k$ the number of parameters.

**Bibliography**

> Ruano, M.V., Ribes, J., Seco, A., Ferrer, J., 2012. An improved sampling strategy based on trajectory design for application of the Morris method to systems with many input factors. Environmental Modelling & Software 37, 103–109. https://doi.org/10.1016/j.envsoft.2012.03.008

> Morris, M.D., 1991. Factorial Sampling Plans for Preliminary Computational Experiments. Technometrics 33, 161–174. https://doi.org/10.1080/00401706.1991.10484804

> Campolongo, F., Cariboni, J., Saltelli, A., 2007. An effective screening design for sensitivity analysis of large models. Environmental Modelling & Software, Modelling, computer-assisted simulations, and mapping of dangerous phenomena for hazard assessment 22, 1509–1518. https://doi.org/10.1016/j.envsoft.2006.10.004


In [6]:
morris_analysis(CONFIGURATION_FILE_MDA, REFERENCE_DESIGN_FILE)

    'data': [{'error_y': {'array': [], 'type': 'data'},
         …

## 2.2 Global sensitivity analysis - Sobol' indices using Saltelli's sampling scheme

The Sobol' method allows to get a representation of the contribution of the inputs to the overall uncertainty in the model output. The space of the uncertain inputs is explored with a Monte Carlo sampling (here Saltelli's scheme, which extends the Sobol' sequence in a way to reduce the error rates in the resulting sensitivity index calculations).

The number of simulation runs required are $m(k+2)$ where $m$ is the number of samples (typically between 100s to 1000s) and $k$ the number of parameters.

**Bibliography**

> Campolongo, F., Saltelli, A., Cariboni, J., 2011. From screening to quantitative sensitivity analysis. A unified approach. Computer Physics Communications 182, 978–988. https://doi.org/10.1016/j.cpc.2010.12.039

> Saltelli, A., 2002. Making best use of model evaluations to compute sensitivity indices. Computer Physics Communications 145, 280–297. https://doi.org/10.1016/S0010-4655(02)00280-1

> Sobol’, I.M., 2001. Global sensitivity indices for nonlinear mathematical models and their Monte Carlo estimates. Mathematics and Computers in Simulation, The Second IMACS Seminar on Monte Carlo Methods 55, 271–280. https://doi.org/10.1016/S0378-4754(00)00270-6


In [7]:
sobol_analysis(CONFIGURATION_FILE_MDA, REFERENCE_DESIGN_FILE)

    'data': [{'error_y': {'array': [], 'type': 'data'},
         …